In [3]:
import datetime as dt

import backtrader as bt
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

from stockdatamanage.db import engine

### 读取数据
读取数据时必须加parse_dates参数，否则backtrader会报错

In [4]:
# 沪深300指数数据
sql = 'select trade_date, close from index_daily where code="399300" and trade_date>="20130101"'
df = pd.read_sql(sql, engine, parse_dates=['trade_date'])
df['high'] = 0
df['low'] = 0
df['open'] = 0
df['volumn'] = 0
df['openinterest'] = 0
df = df[['trade_date', 'open', 'high', 'low', 'close', 'volumn', 'openinterest']]

# 沪深300滚动PE
sql = 'select trade_date, pe from index_dailyindicator where code="399300.SZ" and trade_date>="20130101"'
pedf = pd.read_sql(sql, engine, parse_dates=['trade_date'])
df = df.merge(pedf, how='left', on='trade_date')

df.rename(columns={'trade_date': 'datetime'}, inplace=True)
df.set_index('datetime', inplace=True)
df

,open,high,low,close,volumn,openinterest,pe
datetime,,,,,,,
2013-01-04,0,0,0,2524.41,0,0,10.96
2013-01-07,0,0,0,2535.99,0,0,10.97
2013-01-08,0,0,0,2525.33,0,0,10.90
2013-01-09,0,0,0,2526.13,0,0,10.90
2013-01-10,0,0,0,2530.57,0,0,10.94
...,...,...,...,...,...,...,...
2022-11-08,0,0,0,3749.33,0,0,10.88
2022-11-09,0,0,0,3714.27,0,0,10.79
2022-11-10,0,0,0,3685.69,0,0,10.74


In [5]:
def quantile_rate(x):
    return int(len(x[x<=x.iloc[-1]]) / len(x) * 100)

In [6]:
# 计算PE百分位
df['quantile'] = df['pe'].rolling(window=600).apply(quantile_rate)
df

,open,high,low,close,volumn,openinterest,pe,quantile
datetime,,,,,,,,
2013-01-04,0,0,0,2524.41,0,0,10.96,NaN
2013-01-07,0,0,0,2535.99,0,0,10.97,NaN
2013-01-08,0,0,0,2525.33,0,0,10.90,NaN
2013-01-09,0,0,0,2526.13,0,0,10.90,NaN
2013-01-10,0,0,0,2530.57,0,0,10.94,NaN
...,...,...,...,...,...,...,...,...
2022-11-08,0,0,0,3749.33,0,0,10.88,1.0
2022-11-09,0,0,0,3714.27,0,0,10.79,1.0
2022-11-10,0,0,0,3685.69,0,0,10.74,1.0


In [7]:
df['q0'] = df.pe.rolling(600).quantile(0)
df['q25'] = df.pe.rolling(600).quantile(.25)
df['q75'] = df.pe.rolling(600).quantile(.75)
df['q100'] = df.pe.rolling(600).quantile(1)
df

,open,high,low,close,volumn,openinterest,pe,quantile,q0,q25,q75,q100
datetime,,,,,,,,,,,,
2013-01-04,0,0,0,2524.41,0,0,10.96,NaN,NaN,NaN,NaN,NaN
2013-01-07,0,0,0,2535.99,0,0,10.97,NaN,NaN,NaN,NaN,NaN
2013-01-08,0,0,0,2525.33,0,0,10.90,NaN,NaN,NaN,NaN,NaN
2013-01-09,0,0,0,2526.13,0,0,10.90,NaN,NaN,NaN,NaN,NaN
2013-01-10,0,0,0,2530.57,0,0,10.94,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-08,0,0,0,3749.33,0,0,10.88,1.0,10.28,12.4075,14.76,17.44
2022-11-09,0,0,0,3714.27,0,0,10.79,1.0,10.28,12.4075,14.76,17.44
2022-11-10,0,0,0,3685.69,0,0,10.74,1.0,10.28,12.4075,14.76,17.44


In [8]:
df[~df['quantile'].isna()]

,open,high,low,close,volumn,openinterest,pe,quantile,q0,q25,q75,q100
datetime,,,,,,,,,,,,
2017-05-09,0,0,0,3352.53,0,0,12.65,37.0,9.14,12.0500,13.6425,19.00
2017-05-10,0,0,0,3337.70,0,0,12.61,37.0,9.32,12.0500,13.6425,19.00
2017-05-11,0,0,0,3356.65,0,0,12.70,38.0,9.44,12.0650,13.6425,19.00
2017-05-12,0,0,0,3385.38,0,0,12.83,41.0,9.60,12.0775,13.6425,19.00
2017-05-15,0,0,0,3399.19,0,0,12.87,44.0,9.71,12.0875,13.6425,19.00
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-08,0,0,0,3749.33,0,0,10.88,1.0,10.28,12.4075,14.7600,17.44
2022-11-09,0,0,0,3714.27,0,0,10.79,1.0,10.28,12.4075,14.7600,17.44
2022-11-10,0,0,0,3685.69,0,0,10.74,1.0,10.28,12.4075,14.7600,17.44


In [9]:
df[['pe', 'q0', 'q25', 'q75', 'q100']].plot()
plt.show()

C:\Users\who8736\AppData\Local\Temp\ipykernel_14732\3446796389.py:2: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


### 定义策略

In [15]:
class TestStrategy(bt.Strategy):
    params = (
        ('pe', -1),
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.datape = self.data.pe
        self.dataclose = self.data.close

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log(f'PE, {self.datape[0]:.2f}')
        self.log(f'Close, {self.dataclose[0]:.2f}')

### 数据格式类

In [11]:
class IndexData(bt.feeds.PandasData):
    lines = ('pe', )
    params=(
        ('pe', -1),
        
    )

### 回测初始化

In [86]:
# df.index = pd.to_datetime(df.index)
# df.index[0]
# df.columns
# df.head()

df.iloc[:, [-1]]

,pe
datetime,
2013-01-04,10.96
2013-01-07,10.97
2013-01-08,10.90
2013-01-09,10.90
2013-01-10,10.94
...,...
2022-11-08,10.88
2022-11-09,10.79
2022-11-10,10.74


In [16]:
cerebro = bt.Cerebro()

# 导入数据
datafeed = IndexData(dataname=df, fromdate=dt.datetime(2013, 1, 1),
                                todate=dt.datetime(2022,1,1))
# datafeed = bt.feeds.PandasData(dataname=df, fromdate=datetime.datetime(2019,1,2), todate=datetime.datetime(2021,1,28))
cerebro.adddata(datafeed, name='hs300') # 通过 name 实现数据集与股票的一一对应
cerebro.broker.setcash(1000000)

# 增加策略
cerebro.addstrategy(TestStrategy)
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 1000000.00
2013-01-04, PE, 10.96
2013-01-04, Close, 2524.41
2013-01-07, PE, 10.97
2013-01-07, Close, 2535.99
2013-01-08, PE, 10.90
2013-01-08, Close, 2525.33
2013-01-09, PE, 10.90
2013-01-09, Close, 2526.13
2013-01-10, PE, 10.94
2013-01-10, Close, 2530.57
2013-01-11, PE, 10.78
2013-01-11, Close, 2483.23
2013-01-14, PE, 11.11
2013-01-14, Close, 2577.73
2013-01-15, PE, 11.15
2013-01-15, Close, 2595.86
2013-01-16, PE, 11.05
2013-01-16, Close, 2577.09
2013-01-17, PE, 10.93
2013-01-17, Close, 2552.76
2013-01-18, PE, 11.08
2013-01-18, Close, 2595.44
2013-01-21, PE, 11.09
2013-01-21, Close, 2610.90
2013-01-22, PE, 11.05
2013-01-22, Close, 2596.90
2013-01-23, PE, 11.08
2013-01-23, Close, 2607.46
2013-01-24, PE, 11.08
2013-01-24, Close, 2582.75
2013-01-25, PE, 11.01
2013-01-25, Close, 2571.67
2013-01-28, PE, 11.28
2013-01-28, Close, 2651.86
2013-01-29, PE, 11.33
2013-01-29, Close, 2675.87
2013-01-30, PE, 11.46
2013-01-30, Close, 2688.71
2013-01-31, PE, 11.49
2013-01-31

In [101]:
# a = dftmp.date_column[0]
a.date()

datetime.date(2010, 11, 12)